In [2]:
import copy
import networkx as nx
import pandas as pd 
from collections import defaultdict, OrderedDict

from bondnet.utils import to_path, int_atom, xyz2mol
from bondnet.core.molwrapper import rdkit_mol_to_wrapper_mol, create_wrapper_mol_from_atoms_and_bonds
from pymatgen.analysis.graphs import _isomorphic
from bondnet.core.reaction_collection import get_atom_bond_mapping
from bondnet.dataset.mg_barrier import process_species_rdkit

path_mg_data = "/home/santiagovargas/Documents/Dataset/mg_dataset/"
path_json = path_mg_data + "20220613_reaction_data.json"
mg_df = pd.read_json(path_json)

In [12]:
row = mg_df.iloc[100]

species_reactant = [int_atom(i['name']) for i in row['reactant_molecule_graph']["molecule"]["sites"]]
species_products = [int_atom(i['name']) for i in row['product_molecule_graph']["molecule"]["sites"]]
coords_reactant = [i["xyz"] for i in row['reactant_molecule_graph']["molecule"]["sites"]]
coords_products = [i["xyz"] for i in row['product_molecule_graph']["molecule"]["sites"]]
charge = row['charge']
id = str(row["reactant_id"])
free_energy = row['product_free_energy']

reactant_mol = xyz2mol(atoms = species_reactant, 
    coordinates = coords_reactant, 
    charge = charge,
    )
reactant_wrapper = rdkit_mol_to_wrapper_mol(reactant_mol[0],
    charge = charge,
    free_energy=free_energy, 
    identifier = id
    )
#print(reactant_wrapper.bonds) 
#print(species_reactant)
#print(species_products)

xyz_list = []
for i in reactant_mol[0].GetAtoms():
    xyz_list.append(i.GetAtomicNum())
#print(xyz_list)    

rxn_alt_double_product = process_species(row)





77760_C3F2H8N3O2_-1_-17219.301864222605_index-1
     RDKit          3D

  0  0  0  0  0  0  0  0  0  0999 V3000
M  V30 BEGIN CTAB
M  V30 COUNTS 18 16 0 0 0
M  V30 BEGIN ATOM
M  V30 1 C 0.142598 -0.120857 -0.054459 1
M  V30 2 H 1.67389 1.33708 0.025509 2
M  V30 3 C 0.568922 1.36509 -0.066604 3
M  V30 4 C 0.546988 -0.784415 -1.36453 4
M  V30 5 H 0.08815 -0.297059 -2.23251 5
M  V30 6 H 1.64288 -0.724906 -1.42771 6
M  V30 7 H 0.240421 -1.84057 -1.32507 7
M  V30 8 N 0.678427 -0.735276 1.10673 8
M  V30 9 H 1.71157 -0.510586 1.06824 9
M  V30 10 H 0.583416 -1.74365 1.03031 10
M  V30 11 N 0.062325 2.03969 1.22182 11 CHG=1
M  V30 12 O -1.02233 2.59542 1.17346 12 CHG=-1
M  V30 13 O 0.769823 2.01831 2.19787 13
M  V30 14 N 0.102535 2.05562 -1.21458 14
M  V30 15 H 0.508805 2.97952 -1.29345 15
M  V30 16 H -0.910029 2.13385 -1.20627 16
M  V30 17 F -1.27496 -0.163275 0.001631 17
M  V30 18 F 3.08818 0.06287 0.366937 18 CHG=-1
M  V30 END ATOM
M  V30 BEGIN BOND
M  V30 1 1 1 3
M  V30 2 1 1 4
M  V30 3 1 1 8

In [41]:
works_count = 0
works_single_product = 0
single_product_count = 0
double_product_count = 0 
works_double_product = 0 
from tqdm import tqdm 

for i in tqdm(range(200)):
    row = mg_df.iloc[i]
    """g_prod = nx.Graph(row["product_bonds"])
    g_react = nx.Graph(row["reactant_bonds"])
    g_prod_2 = nx.Graph(row["product_bonds_nometal"])
    g_react_2 = nx.Graph(row["reactant_bonds_nometal"])
    if(nx_graph_atom_mapping(g_prod, g_react) == None):
        if(nx_graph_atom_mapping(g_prod_2, g_react_2) == None):
            pass
        else: works_count += 1 
    else: works_count += 1"""
    species_reactant = [int_atom(i['name']) for i in row['reactant_molecule_graph']["molecule"]["sites"]]
    species_products = [int_atom(i['name']) for i in row['product_molecule_graph']["molecule"]["sites"]]
    coords_reactant = [i["xyz"] for i in row['reactant_molecule_graph']["molecule"]["sites"]]
    coords_products = [i["xyz"] for i in row['product_molecule_graph']["molecule"]["sites"]]

    r = create_wrapper_mol_from_atoms_and_bonds(
        species_reactant, coords_reactant, row["reactant_bonds"], charge=0
    )
    p = create_wrapper_mol_from_atoms_and_bonds(
        species_products, coords_products, row["product_bonds"], charge=0
    )
    r2 = create_wrapper_mol_from_atoms_and_bonds(
        species_reactant, coords_reactant, row["reactant_bonds_nometal"], charge=0
    )
    p2 = create_wrapper_mol_from_atoms_and_bonds(
        species_products, coords_products, row["product_bonds_nometal"], charge=0
    )
  
    rxn_alt_double_product = []

    if(type(rxn_alt_double_product) == list):
        products = p
        product_s = copy.deepcopy(products)
        #original.break_edge(bond[0], bond[1], allow_reverse=True)
        components = nx.weakly_connected_components(product_s.graph)
        subgraphs = [product_s.graph.subgraph(c) for c in components]
        subgraphs_count = int(len(subgraphs))

    else: 
        products = rxn_alt_double_product.products
        subgraphs_count = int(len(products))

    if(subgraphs_count==2):double_product_count+=1
    else: 
        single_product_count+=1
        mapping = 

    """
    if(subgraphs_count == 2):
        if(nx_graph_atom_mapping(r.mol_graph.graph, p.mol_graph.graph) == None):
            if(nx_graph_atom_mapping(r2.mol_graph.graph, p2.mol_graph.graph) == None):
                if(nx_graph_atom_mapping(r.mol_graph.graph, p2.mol_graph.graph) == None):
                    if(nx_graph_atom_mapping(r2.mol_graph.graph, p.mol_graph.graph) == None):
                        pass
                    else: 
                        works_count += 1
                        works_double_product+=1
                else: 
                    works_count += 1
                    works_double_product+=1
            else: 
                works_count += 1 
                works_double_product+=1
        else: 
            works_count += 1
            works_double_product+=1
        if(nx_graph_atom_mapping(r.mol_graph.graph, p.mol_graph.graph) == None):
            if(nx_graph_atom_mapping(r2.mol_graph.graph, p2.mol_graph.graph) == None):
                if(nx_graph_atom_mapping(r.mol_graph.graph, p2.mol_graph.graph) == None):
                    if(nx_graph_atom_mapping(r2.mol_graph.graph, p.mol_graph.graph) == None):
                        pass
                    else: 
                        works_count += 1
                        works_double_product+=1
                else: 
                    works_count += 1
                    works_double_product+=1
            else: 
                works_count += 1 
                works_double_product+=1
        else: 
            works_count += 1
            works_double_product+=1
   
    else:"""


100%|██████████| 200/200 [00:00<00:00, 260.51it/s]


In [42]:
element_dict = {"H": 1, "Li": 3, "Be": 4, "B": 5, "C": 6, "N": 7, "O": 8, 
"F": 9, 'Ca': 20, 'Na': 11, 'Cl': 17, 'P':15, 'Mg': 12 }
preserve = 0
for i in tqdm(range(10)):
    row = mg_df.iloc[i]
    """g_prod = nx.Graph(row["product_bonds"])
    g_react = nx.Graph(row["reactant_bonds"])
    g_prod_2 = nx.Graph(row["product_bonds_nometal"])
    g_react_2 = nx.Graph(row["reactant_bonds_nometal"])
    if(nx_graph_atom_mapping(g_prod, g_react) == None):
        if(nx_graph_atom_mapping(g_prod_2, g_react_2) == None):
            pass
        else: works_count += 1 
    else: works_count += 1"""
    species_reactant = [int_atom(i['name']) for i in row['reactant_molecule_graph']["molecule"]["sites"]]
    species_products = [int_atom(i['name']) for i in row['product_molecule_graph']["molecule"]["sites"]]
    coords_reactant = [i["xyz"] for i in row['reactant_molecule_graph']["molecule"]["sites"]]
    coords_products = [i["xyz"] for i in row['product_molecule_graph']["molecule"]["sites"]]
    #r = create_wrapper_mol_from_atoms_and_bonds(
    #    species_reactant, coords_reactant, row["reactant_bonds"], charge=0
    #)

    #p = create_wrapper_mol_from_atoms_and_bonds(
    #    species_products, coords_products, row["product_bonds"], charge=0
    #)
    reactant_mol = xyz2mol(atoms = species_reactant, 
    coordinates = coords_reactant
    )
    reactant_wrapper = rdkit_mol_to_wrapper_mol(reactant_mol[0],
        identifier = '1'
        )

    list_temp = [i['name'] for i in list(reactant_wrapper.mol_graph.as_dict()['molecule']['sites'])]
    list_xyz = [i['xyz'] for i in list(reactant_wrapper.mol_graph.as_dict()['molecule']['sites'])]
    list_atomic = [element_dict[i] for i in list_temp]
    
    if(list_atomic == species_reactant): preserve += 1
    if(list_xyz == coords_reactant): preserve += 1

print(preserve)


100%|██████████| 10/10 [00:00<00:00, 61.88it/s]

20


13
